In [2]:
import csv
import re
from argparse import ArgumentParser
from collections import defaultdict
from os.path import splitext
from operator import itemgetter

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font
from openpyxl.utils import get_column_letter


In [7]:
input_path = '/content/Mumbai_event_data_20240201.xlsx'
output_path = '/content/Summary_Mumbai_event_data_20240201.xlsx'

category_threshold = 100
date_format = r"\d{1,2}[-/]\d{1,2}[-/]\d{2}(\d{2})?"
date_time_format = r"\d{1,2}[-/]\d{1,2}[-/]\d{2}(\d{2})?[- ]\d{2}:\d{2}:\d{2}(\.\d{1,6})?"
ignore_values = ['?', 'N/A']
sheet_name = None
num_samples = 3

In [6]:
def summarize_excel(input_path, output_path, category_threshold, date_format, date_time_format, ignore_values, sheet_name, num_samples):
    date_regex = re.compile(date_format)
    date_time_regex = re.compile(date_time_format)

    wb = load_workbook(input_path)
    ws = wb[sheet_name] if sheet_name else wb.active

    headers = [cell.value for cell in next(ws.iter_rows(min_row=1, max_row=1))]
    data_columns = {header: defaultdict(int) for header in headers}
    all_dates = defaultdict(lambda: True)
    all_date_times = defaultdict(lambda: True)

    for row in ws.iter_rows(min_row=2):
        for header, cell in zip(headers, row):
            value = str(cell.value)
            if value not in ignore_values:
                data_columns[header][value] += 1
                all_dates[header] &= bool(date_regex.fullmatch(value))
                all_date_times[header] &= bool(date_time_regex.fullmatch(value))

    summary_ws = wb.create_sheet("Summary")
    samples_ws = wb.create_sheet("Samples")


    summary_ws.append(headers)
    for col_idx, header in enumerate(headers, start=1):
        column_data = data_columns[header]
        if len(column_data) <= category_threshold:
            for row_idx, (value, count) in enumerate(column_data.items(), start=2):
                summary_ws.cell(row=row_idx, column=col_idx).value = f"{value} ({count})"
        else:
            summary_ws.cell(row=2, column=col_idx).value = "Various Values"


    for row_idx, header in enumerate(headers, start=1):
        samples_ws.cell(row=row_idx, column=1).value = header
    for col_idx in range(2, num_samples + 2):
        for row_idx, header in enumerate(headers, start=1):
            samples_ws.cell(row=row_idx, column=col_idx).value = ws.cell(row=col_idx, column=row_idx).value


    for sheet in [summary_ws, samples_ws]:
        for column_cells in sheet.columns:
            length = max(len(str(cell.value)) for cell in column_cells)
            sheet.column_dimensions[get_column_letter(column_cells[0].column)].width = length
    wb.save(output_path)


In [8]:

summarize_excel(input_path, output_path, category_threshold, date_format, date_time_format, ignore_values, sheet_name, num_samples)